# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
from secret import IEX_CLOUD_API_TOKEN
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()[0]
data

{'avgTotalVolume': 52599559,
 'calculationPrice': 'iexlasttrade',
 'change': 1.03,
 'changePercent': 0.00526,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 196.93,
 'iexAskSize': 200,
 'iexBidPrice': 99.76,
 'iexBidSize': 102,
 'iexClose': 196.92,
 'iexCloseTime': 1703019599964,
 'iexLastUpdated': 1703019599964,
 'iexMarketPercent': 0.01868520122382501,
 'iexOpen': 196.13,
 'iexOpenTime': 1702996200939,
 'iexRealtimePrice': 196.92,
 'iexRealtimeSize': 200,
 'iexVolume': 759717,
 'lastTradeTime': 1703019599964,
 'latestPrice': 196.92,
 'latestSource': 'IEX Last Trade',
 'latestTime': 'December 19, 2023',
 'latestUpdate': 1703019599964,
 'latestVolume': None,
 'low': None,
 'lowSource': 'IEX real

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
ratio = data['peRatio']
ratio

32.12

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [27]:
dataframe = pd.DataFrame(columns=my_columns)
import time

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    response = requests.get(batch_api_call_url)
    if (response.status_code != 200):
        print("Problem")
        break
    data = response.json()
    symbols = symbol_string.split(',')
    for i in range(len(symbols)):
        symbol = symbols[i]
        price = data[i]['latestPrice']
        peRatio = data[i]['peRatio']
        row = pd.DataFrame([[symbol, price, peRatio, 'N/A']],columns=my_columns)
        dataframe = pd.concat([dataframe, row], axis=0, ignore_index=True)
    time.sleep(0.2)
print('done')    
        
    

/var/folders/13/07ny0fkn4db5gc851jjdw9x00000gn/T/ipykernel_17929/3793681074.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, row], axis=0, ignore_index=True)


done


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [28]:
dataframe.sort_values(ascending=True, by="Price-to-Earnings Ratio", inplace=True)
dataframe = dataframe[dataframe['Price-to-Earnings Ratio'] > 0]
dataframe.reset_index(inplace=True)
dataframe = dataframe[:50]
len(dataframe) 
dataframe

,index,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,71,BRK.B,361.800,0.01,N/A
1,192,FRC,3.510,0.43,N/A
2,137,DISH,4.870,2.45,N/A
3,159,EMR,95.740,4.18,N/A
4,468,VLO,133.600,4.55,N/A
5,503,ZION,43.950,4.69,N/A
6,454,UAL,42.420,4.92,N/A
7,203,GM,35.870,5.05,N/A
8,23,ALB,148.930,5.28,N/A
9,1,AAL,14.310,5.34,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [18]:
class CheapPortfolioException(Exception):
    pass

def getPortfolioSize():
    val = 0
    while True:
        val = input("Enter the value of your portfolio: ")
        try:
            val = float(val)
            if val <= 0:
                raise CheapPortfolioException("That value is too small")
            return val
        except ValueError:
            print("That was not a number")
        except CheapPortfolioException:
            print("That number was too small")

portfolioSize = getPortfolioSize()
positionSize = portfolioSize/len(dataframe.index)

Enter the value of your portfolio:  1000000


Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [29]:
for row in dataframe.index:
    dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(positionSize/dataframe.loc[row, 'Price'])

dataframe

,index,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,71,BRK.B,361.800,0.01,55
1,192,FRC,3.510,0.43,5698
2,137,DISH,4.870,2.45,4106
3,159,EMR,95.740,4.18,208
4,468,VLO,133.600,4.55,149
5,503,ZION,43.950,4.69,455
6,454,UAL,42.420,4.92,471
7,203,GM,35.870,5.05,557
8,23,ALB,148.930,5.28,134
9,1,AAL,14.310,5.34,1397


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [36]:
symbol = "AAPL"
quoteUrl = f"https://api.iex.cloud/v1/data/CORE/QUOTE/{symbol}?token={IEX_CLOUD_API_TOKEN}"
statsUrl = f"https://api.iex.cloud/v1/data/CORE/ADVANCED_STATS/{symbol}?token={IEX_CLOUD_API_TOKEN}"

quoteResponse = requests.get(quoteUrl)
statsResponse = requests.get(statsUrl)

if (quoteResponse.status_code != 200 or statsResponse.status_code != 200):
    print("Fail")

quoteData = quoteResponse.json()[0]
statsData = statsResponse.json()[0]

peRatio = quoteData['peRatio']
pbRatio = statsData['priceToBook']
psRatio = statsData['priceToSales']
enterpriseValue = statsData['enterpriseValue']
EBITDA = statsData['EBITDA']

evebitda = enterpriseValue/EBITDA

grossProfit = statsData['grossProfit']
evgp = enterpriseValue/grossProfit

print(peRatio)
print(pbRatio)
print(psRatio)
print(enterpriseValue)
print(EBITDA)
print(evebitda)
print(grossProfit)
print(evgp)


32.12
49.02
7.95
3096161589280
125820000000
24.607865119058975
169148000000
18.304452841771703


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [60]:
rv_columns = [
    'Ticker', 
    'Price', 
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio', 
    'PE Percentile', 
    'Price-to-Book Ratio', 
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rvdf = pd.DataFrame(columns=rv_columns)

In [61]:
for symbol_string in symbol_strings:
    quoteUrl = f"https://api.iex.cloud/v1/data/CORE/QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}"
    statsUrl = f"https://api.iex.cloud/v1/data/CORE/ADVANCED_STATS/{symbol_string}?token={IEX_CLOUD_API_TOKEN}"
    
    quoteResponse = requests.get(quoteUrl)
    statsResponse = requests.get(statsUrl)
    
    if (quoteResponse.status_code != 200 or statsResponse.status_code != 200):
        print("Fail")
    
    quoteData = quoteResponse.json()
    statsData = statsResponse.json()
    symbols = symbol_string.split(',')
    for i in range(len(symbols)):
        if(len(quoteData[i]) == 0 or len(statsData[i]) == 0):
            continue
        symbol = symbols[i]   
        price = quoteData[i]['latestPrice']
        peRatio = quoteData[i]['peRatio']
        pbRatio = statsData[i]['priceToBook']
        psRatio = statsData[i]['priceToSales']
        enterpriseValue = statsData[i]['enterpriseValue']
        EBITDA = statsData[i]['EBITDA']

        evebitda = np.NaN
        if (enterpriseValue != None and EBITDA != None):
            evebitda = enterpriseValue/EBITDA
        else:
            print(f"Cannot calculate EV/EBITDA for {symbol}")
        
        grossProfit = statsData[i]['grossProfit']
        evgp = np.NaN
        if (grossProfit != None and enterpriseValue != None):
            evgp = enterpriseValue/grossProfit
        else:
            print(f"Cannot calculate EV/GP for {symbol}")
        row = pd.DataFrame([[symbol, price, 'N/A', peRatio, 'N/A', pbRatio, 'N/A', psRatio, 'N/A', evebitda, 'N/A', evgp, 'N/A', 'N/A']],columns=rv_columns)
        rvdf = pd.concat([rvdf, row], axis=0, ignore_index=True)
    time.sleep(0.4)
print('done')    

/var/folders/13/07ny0fkn4db5gc851jjdw9x00000gn/T/ipykernel_17929/145776649.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rvdf = pd.concat([rvdf, row], axis=0, ignore_index=True)


Cannot calculate EV/EBITDA for ABMD
Cannot calculate EV/GP for ABMD
Cannot calculate EV/EBITDA for ALXN
Cannot calculate EV/GP for ALXN
Cannot calculate EV/EBITDA for AON
Cannot calculate EV/GP for AON
Cannot calculate EV/EBITDA for ATVI
Cannot calculate EV/GP for ATVI
Cannot calculate EV/EBITDA for CERN
Cannot calculate EV/GP for CERN
Cannot calculate EV/EBITDA for CTL
Cannot calculate EV/GP for CTL
Cannot calculate EV/EBITDA for CTXS
Cannot calculate EV/GP for CTXS
Cannot calculate EV/EBITDA for DISCK
Cannot calculate EV/GP for DISCK
Cannot calculate EV/EBITDA for ETFC
Cannot calculate EV/GP for ETFC
Cannot calculate EV/EBITDA for FISV
Cannot calculate EV/GP for FISV
Cannot calculate EV/EBITDA for FLIR
Cannot calculate EV/GP for FLIR
Cannot calculate EV/EBITDA for FRT
Cannot calculate EV/GP for FRT
Cannot calculate EV/EBITDA for J
Cannot calculate EV/GP for J
Cannot calculate EV/EBITDA for MXIM
Cannot calculate EV/GP for MXIM
Cannot calculate EV/EBITDA for MYL
Cannot calculate EV/GP 

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [62]:
rvdf

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.79,N/A,36.59,N/A,7.24,N/A,5.7500,N/A,24.601491,N/A,11.706774,N/A,N/A
1,AAL,14.30,N/A,5.34,N/A,-1.81,N/A,0.1759,N/A,4.827427,N/A,0.608788,N/A,N/A
2,AAP,60.86,N/A,19.51,N/A,1.36,N/A,0.3195,N/A,8.239828,N/A,1.081394,N/A,N/A
3,AAPL,196.94,N/A,32.13,N/A,49.02,N/A,7.9500,N/A,24.607865,N/A,18.304453,N/A,N/A
4,ABBV,153.60,N/A,42.08,N/A,22.40,N/A,4.9100,N/A,12.661579,N/A,8.779677,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,130.24,N/A,24.76,N/A,-4.47,N/A,5.1800,N/A,19.750547,N/A,8.961242,N/A,N/A
501,ZBH,119.97,N/A,53.08,N/A,1.98,N/A,3.4100,N/A,15.658429,N/A,5.756930,N/A,N/A
502,ZBRA,269.42,N/A,30.04,N/A,4.55,N/A,2.7000,N/A,18.243783,N/A,6.740923,N/A,N/A
503,ZION,43.93,N/A,4.69,N/A,1.31,N/A,1.2800,N/A,3.511192,N/A,1.325088,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [68]:
rvdf[rvdf.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [67]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rvdf[column].fillna(rvdf[column].mean(), inplace=True)

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [72]:
metrics = {
    'Price-to-Earnings Ratio':'PE Percentile', 
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}
for metric in metrics.keys():
    for row in rvdf.index:
        value = rvdf.loc[row, metric]
        rvdf.loc[row, metrics[metric]] = stats.percentileofscore( rvdf[metric], value)/100

rvdf

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.79,N/A,36.59,0.79604,7.24,0.811881,5.7500,0.825743,24.601491,0.869307,11.706774,0.831683,N/A
1,AAL,14.30,N/A,5.34,0.09901,-1.81,0.10297,0.1759,0.011881,4.827427,0.055446,0.608788,0.017822,N/A
2,AAP,60.86,N/A,19.51,0.445545,1.36,0.221782,0.3195,0.041584,8.239828,0.192079,1.081394,0.057426,N/A
3,AAPL,196.94,N/A,32.13,0.726733,49.02,0.976238,7.9500,0.89505,24.607865,0.871287,18.304453,0.952475,N/A
4,ABBV,153.60,N/A,42.08,0.843564,22.40,0.956436,4.9100,0.772277,12.661579,0.419802,8.779677,0.673267,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,130.24,N/A,24.76,0.560396,-4.47,0.09901,5.1800,0.790099,19.750547,0.728713,8.961242,0.685149,N/A
501,ZBH,119.97,N/A,53.08,0.89703,1.98,0.361386,3.4100,0.594059,15.658429,0.556436,5.756930,0.407921,N/A
502,ZBRA,269.42,N/A,30.04,0.681188,4.55,0.676238,2.7000,0.532673,18.243783,0.677228,6.740923,0.491089,N/A
503,ZION,43.93,N/A,4.69,0.091089,1.31,0.210891,1.2800,0.237624,3.511192,0.025743,1.325088,0.083168,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [75]:
from statistics import mean

for row in rvdf.index:
    values = []
    for metric in metrics.keys():
        values.append(rvdf.loc[row, metrics[metric]])
    rvdf.loc[row, "RV Score"] = mean(values)

rvdf

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.79,N/A,36.59,0.79604,7.24,0.811881,5.7500,0.825743,24.601491,0.869307,11.706774,0.831683,0.826931
1,AAL,14.30,N/A,5.34,0.09901,-1.81,0.10297,0.1759,0.011881,4.827427,0.055446,0.608788,0.017822,0.057426
2,AAP,60.86,N/A,19.51,0.445545,1.36,0.221782,0.3195,0.041584,8.239828,0.192079,1.081394,0.057426,0.191683
3,AAPL,196.94,N/A,32.13,0.726733,49.02,0.976238,7.9500,0.89505,24.607865,0.871287,18.304453,0.952475,0.884356
4,ABBV,153.60,N/A,42.08,0.843564,22.40,0.956436,4.9100,0.772277,12.661579,0.419802,8.779677,0.673267,0.733069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,130.24,N/A,24.76,0.560396,-4.47,0.09901,5.1800,0.790099,19.750547,0.728713,8.961242,0.685149,0.572673
501,ZBH,119.97,N/A,53.08,0.89703,1.98,0.361386,3.4100,0.594059,15.658429,0.556436,5.756930,0.407921,0.563366
502,ZBRA,269.42,N/A,30.04,0.681188,4.55,0.676238,2.7000,0.532673,18.243783,0.677228,6.740923,0.491089,0.611683
503,ZION,43.93,N/A,4.69,0.091089,1.31,0.210891,1.2800,0.237624,3.511192,0.025743,1.325088,0.083168,0.129703


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [79]:
rvdf.sort_values('RV Score', ascending = False, inplace=True)
rvdf = rvdf[:50]
rvdf.reset_index(drop = True, inplace = True)
rvdf

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,LLY,579.810,N/A,104.85,0.976238,49.05,0.978218,17.16,0.992079,52.226798,0.978218,22.514477,0.984158,0.981782
1,NVDA,496.040,N/A,65.35,0.934653,37.18,0.974257,27.57,1.0,55.429914,0.980198,39.189832,1.0,0.977822
2,CDNS,274.330,N/A,78.16,0.954455,23.90,0.960396,18.98,0.99802,64.123484,0.986139,21.189510,0.970297,0.973861
3,DXCM,123.290,N/A,139.86,0.982178,21.01,0.946535,14.01,0.964356,91.114732,0.992079,21.638491,0.978218,0.972673
4,NOW,707.710,N/A,91.55,0.962376,20.09,0.942574,17.02,0.990099,120.786530,0.994059,21.292846,0.974257,0.972673
5,VRSK,234.455,N/A,70.20,0.946535,89.49,0.992079,15.40,0.980198,29.237455,0.926733,24.051980,0.99604,0.968317
6,ISRG,335.720,N/A,78.99,0.956436,9.22,0.861386,16.88,0.988119,52.128398,0.976238,23.921028,0.994059,0.955248
7,SNPS,558.650,N/A,70.54,0.948515,13.84,0.920792,14.56,0.976238,60.753775,0.982178,18.070425,0.944554,0.954455
8,IDXX,545.700,N/A,55.68,0.906931,34.57,0.972277,12.50,0.955446,38.191314,0.962376,21.190827,0.972277,0.953861
9,ADBE,604.640,N/A,54.47,0.90099,17.29,0.931683,14.44,0.974257,40.866397,0.968317,16.200801,0.922772,0.939604


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [83]:
portfolioSize = getPortfolioSize()
positionSize = portfolioSize/len(rvdf.index)

Enter the value of your portfolio:  2500000


In [85]:
for row in rvdf.index:
    rvdf.loc[row, 'Number of Shares to Buy'] = math.floor(positionSize/rvdf.loc[row, 'Price'])


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [87]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rvdf.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [88]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [89]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [90]:
writer.close()